# --- Day 20: Race Condition ---

In [120]:
# --- Part One and Part Two ---

from typing import List, Dict, Tuple
from collections import deque, defaultdict

filename = "input.txt"
#filename = "test.txt" # decomment to test

class Racemap:

    def __init__(self, filename: str):
        self.height :int = 0
        self.size: Tuple[int,int] = ()
        self.start : Tuple[int, int] = None
        self.end : Tuple[int, int] = None
        self.cheats = defaultdict(int)
        self.path: Dict[Tuple[int,int]: int] = {}
        self.racemap = self.load(filename)
        self._map()

    def load(self, filename: str) -> List[List[str]]:
        with open(filename, 'rt') as file:
            return [list(c) for c in [l.strip() for l in file]]

    def _map(self):
        # map racemap caracteristics : size and possible cheats
        self.size = (len(self.racemap[0]),len(self.racemap))
        for y, row in enumerate(self.racemap):
            for x, cell in enumerate(row):
                if cell == "#": 
                    continue
                if cell == "S":
                    self.start = (x,y)
                if cell == "E":
                    self.end = (x,y)


    def bsf_path_find(self):
        """
        Breadth-First Search : find minimum path (useable if all steps have same cost)
        """
        queue = deque([(self.start, 0)]) # BFS queue : (pos(x,y) , picoseconds)
        path = {}
        while queue:
            (x,y), ps = queue.popleft()
            # add cell to path
            path[(x,y)] = ps 
            # If 'end' position is reached, return the path
            if (x, y) == self.end: 
                self.path = path
                return self.path
            # Explore the neighbors
            for nx, ny in [(x-1, y), (x+1 , y), (x, y-1),(x, y+1)]:
                # Check if neighbor is valid and not visited
                if self.racemap[ny][nx] != "#" and (nx, ny) not in path:
                    queue.appendleft(((nx, ny), ps +1))
        # If path not found
        return None

    def _possible_cheats(self, x, y, cheat_len):
        ret = []
        # test for all possible lenghts from 2 to max_len
        for dx in range(cheat_len +1):
            dy = cheat_len - dx
            ret.extend(set([(x + dx, y + dy),(x + dx, y - dy),(x - dx, y + dy),(x - dx, y - dy)]))
        return ret

    def map_cheats(self, max_len = 2):
        """
        Counts hoew many cheats are for each timesave value
        """
        self.cheats.clear()
        for y in range(self.size[1]):
            for x in range(self.size[0]):
                if (x,y) not in self.path: continue
                for cheat_len in range (2, max_len+1):
                    for x2, y2 in self._possible_cheats( x, y, cheat_len):
                        if (x2, y2) not in self.path: continue
                        time_saved = self.path[(x2, y2)] - self.path[(x, y)] - cheat_len
                        if time_saved > 0:
                            self.cheats[time_saved] += 1                   

    def __str__(self):
        retstr = ""
        for row in self.racemap:
            retstr += "".join(row) + "\n"
        return retstr



# ---- main ----
racemap = Racemap(filename)

# print(racemap)
path = racemap.bsf_path_find()
print (f"Normal race (no cheat) is {len(path)-1} picoseconds.")

racemap.map_cheats(2)
cheats = racemap.cheats
res1 = sum([y for x, y in cheats.items() if x >= 100])
print("With a cheat shortcut of max 2 steps")
print(f"The solution for part 1 is: {res1}")

racemap.map_cheats(20)
cheats = racemap.cheats
res2 = sum([y for x, y in cheats.items() if x >= 100])
print("With a cheat shortcut of max 20 steps")
print(f"The solution for part 2 is: {res2}")


Normal race (no cheat) is 9336 picoseconds.
With a cheat shortcut of max 2 steps
The solution for part 1 is: 1375
With a cheat shortcut of max 20 steps
The solution for part 2 is: 983054
